In [ ]:
import pandas as pd
import os
from pycaret.classification import load_model, predict_model
import datetime

In [ ]:
#Working directory

#df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx'',sheet_name='can_tintorer', engine='openpyxl')
#df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx'',sheet_name='encinsasola', engine='openpyxl') 
df = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx',sheet_name='aliste', engine='openpyxl')
df.info()


In [ ]:
#models
 modelo = load_model(model_path)